======================== Import Packages ==========================

In [ ]:
import sys, os, pdb, glob
import numpy as np
from astropy.table import Table, join
from astroquery.vizier import Vizier
import calc_dust_masses
import warnings
from astropy.logger import AstropyWarning
warnings.filterwarnings('ignore', category=AstropyWarning)

===================== Define Functions ===================

In [ ]:
def get_data(catalog, join_key='Name'):

    """
    PURPOSE:    Get data from literature with Vizier

    INPUT:      catalog = ctalog name on Vizier (str)
                join_key = column header to join tables, if multiple (str; optional)

    OUTPUT:     t = data table (AstroPy Table)

    """

    ### GET FULL CATALOG (ALL COLUMNS, ALL ROWS)
    viz = Vizier(catalog=catalog, columns=['**'])
    viz.ROW_LIMIT = -1
    tv = viz.get_catalogs(catalog)

    ### IF MULTIPLE TABLES, JOIN THEN
    for i, val in enumerate(tv.keys()):
        if i == 0:
            t = tv[val]
        else:
            tt = tv[val]
            if join_key in tt.columns:
                t = join(t, tt, join_type='inner', keys=join_key)

    return t

In [ ]:
def fix_table(t, region, col_name, col_spt, col_flux, col_eflux, col_mstar, dist, obs_wave, col_lflux=None, log_mstar=False):

    """
    PURPOSE:    Make tables for different regions consistent

    INPUT:      t = table from literature (AstroPy Table)
                region = name of region (str)
                col_X = column name for data values (str)
                dist = distance to source in pc (float arr)
                obs_wave = wavelength of observation in microns (float)
                col_lflux = column name if fluxes for non-detections are upper limits (str; optional)
                log_mstar = boolean flag if stellar mass is log (boolean; optional)

    OUTPUT:     t = fixed data table (AstroPy Table)

    """

   
    ### FOR TAURUS, DO SOME EXTRA FIXING
    t['ObsWave'] = obs_wave
    if region == 'tau':
        ind_1300 = np.where(t['Notes'] == 'e, m')
        t[col_flux][ind_1300] = t['F1.3'][ind_1300]
        t[col_lflux][ind_1300] = t['l_F1.3'][ind_1300]
        t['ObsWave'][ind_1300] = 1300.
        t[col_flux] *= 1e3
        t[col_spt] = [x.replace(')', '').replace('(', '').split('+/-')[0].split('-')[0] for x in t['SpT']]
 
    ### FIX COLUMN NAMES
    t[col_name].name = 'Name'
    t[col_spt].name = 'SpT'
    t[col_mstar].name = 'Mstar'
    t[col_flux].name = 'Flux'
    t[col_eflux].name = 'e_Flux'
    t['Dist'] = dist

    ### FLAG (NON-)DETECTIONS & REPLACE WITH 3-SIGMA UPPER LIMITS
    ### USE 3x ERROR IF TRUE ERRORS REPORTED
    if col_lflux is None:
        t['Det'] = np.repeat(0, len(t))
        t['Det'][t['Flux'] / t['e_Flux'] >= 3.0] = 1
        t['Flux'][np.where(t['Det'] == 0)] = 3.0 * t['e_Flux'][np.where(t['Det'] == 0)]

    ### USE REPORTED FLUX IF UPPER LIMITS PROVIDED
    else:
        t['Det'] = np.repeat(1, len(t))
        t['Det'][np.where(t[col_lflux] == '<')] = 0

    ### ONLY KEEP STARS > 0.1 SOLAR MASS
    if log_mstar:
        t['Mstar'] = 10**t['Mstar']
    t = t[t['Mstar'] >= 0.1]

    ### FOR UPPER SCO, ONLY KEEP "PRIMORDIAL" DISKS 
    ### TO MATCH SAMPLES OF LUPUS & TAURUS
    if region == 'usc':
        t = t[np.where( (t['Type'] == 'Full') | (t['Type'] == 'Transitional') | (t['Type'] == 'Evolved'))] 

    ### CALCULATE DUST MASSES USING SAME METHOD AS LUPUS
    mdust = []
    for i, val in enumerate(t):
        mdust.append(calc_dust_masses.get_dustmass(1.0, t['ObsWave'][i], t['Flux'][i], t['Dist'][i], 20.))
    t['MDust'] = mdust

    return t['Name', 'SpT', 'Dist', 'Mstar', 'Flux', 'ObsWave', 'Det', 'MDust']

========================== Code ==========================

In [ ]:
### GET TAURUS DATA
TT = get_data("J/ApJ/771/129/")
TT = fix_table(TT, 'tau', 'Name', 'SpT', 'F0.89', 'e_F0.89', 'logM_3', np.repeat(140., len(TT)), 890., col_lflux='l_F0.89', log_mstar=True)
TT.write('../output/data_tau.txt', format='ascii.ipac')

In [ ]:
### GET LUPUS DATA
TL = get_data("J/ApJ/828/46")
TL = fix_table(TL, 'lup', 'Name', 'SpT', 'F890', 'e_F890', 'Mass', TL['Dist'], 890.)
TL.write('../output/data_lup.txt', format='ascii.ipac')

In [ ]:
### GET UPPER SCO DATA
TU = get_data("J/ApJ/827/142")
TU = fix_table(TU, 'usc', '_2MASS', 'SpT', 'Snu', 'e_Snu', 'logM', np.repeat(145., len(TU)), 880., log_mstar=True)
TU.write('../output/data_usc.txt', format='ascii.ipac')